In [216]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import json
import time

In [454]:
# Set up Selenium and ChromeDriver
chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
# chrome_options.add_argument("--headless")  # Uncomment to run headless


# Path to your ChromeDriver
driver_path = r"C:/Users/admin/Downloads/chromedriver.exe"

# Create a Service object
service = Service(driver_path)

# Initialize WebDriver
driver = webdriver.Chrome(service=service, options=chrome_options)

# Load the item IDs from the JSON file
with open('IDs.json', 'r') as f:
    item_ids = json.load(f)

# Base URL pattern
base_url = "https://www.ebay.com/itm/"

# List to store scraped data
scraped_data = []

for item_id in item_ids:
    # Create dynamic link
    link = f"{base_url}{item_id}"
    print(f"Opening link: {link}")
    driver.get(link)

    # Wait for the page to load
    try:
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".x-item-title__mainTitle"))
        )
        print(f"Page loaded successfully: {link}")

        # Get the item title
        try:
            title_element = driver.find_element(By.CSS_SELECTOR, ".x-item-title__mainTitle")
            price_element = driver.find_element(By.CSS_SELECTOR, ".x-price-primary")
            title = title_element.text
            price = price_element.text

           
        except:
            title = 'No title found'
            price = 'No price found'

        labels = [] 
        values = []
        try:
            label_element = driver.find_elements(By.CSS_SELECTOR, ".ux-labels-values__labels-content")
            value_element = driver.find_elements(By.CSS_SELECTOR, ".ux-labels-values__values-content")
            for div in label_element:
                if div.text.strip() != 'Payments:' and div.text.strip() != '':
                    labels.append(str(div.text).replace('\n', ' '))
            for div in value_element:
                if div.text.__contains__("International shipment of items may be subject to customs processing and additional charges"):
                    continue
                if div.text.strip() != '':
                    values.append(str(div.text).replace('\n', ' '))

            
        except:
            status = 'No status found'
        

        # Scroll down the page to locate and click "See all feedback" button
        try:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(3)
            feedback_button = WebDriverWait(driver, 20).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "fdbk-detail-list__tabbed-btn.fake-btn.fake-btn--large.fake-btn--secondary"))
            )
            feedback_button.click()
            print(f"Clicked 'See all feedback' button for {item_id}")

            # Wait for feedback section to load and scrape feedback comments
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "fdbk-container__details__comment"))
            )
            feedback_elements = driver.find_elements(By.CSS_SELECTOR, ".fdbk-container__details__comment > span")
            feedback_comments = [element.text for element in feedback_elements]

            # If no feedback is found, add placeholder
            if not feedback_comments:
                feedback_comments = ['No feedback found']

        except Exception as e:
            print(f"Error scraping feedback for {item_id}: {e}")
            feedback_comments = ['No feedback found']

        # Append data to list
        scraped_data.append({
            'Item ID': item_id,
            'Title': title,
            'Price':price,
            'Labels':labels,
            'Values':values,
            'Feedback': feedback_comments
        })

    except Exception as e:
        print(f"Error loading item page for {item_id}: {e}")

# Close the browser
driver.quit()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(scraped_data)

# Save the data to a CSV file with the correct column names
csv_file_name = 'feedback_dat.csv'
df.to_csv(csv_file_name, index=False, encoding='utf-8')

print(f"Scraping completed and data saved to {csv_file_name}")

Opening link: https://www.ebay.com/itm/276429448744
Page loaded successfully: https://www.ebay.com/itm/276429448744
Clicked 'See all feedback' button for 276429448744
Opening link: https://www.ebay.com/itm/256193691993
Page loaded successfully: https://www.ebay.com/itm/256193691993
Clicked 'See all feedback' button for 256193691993
Opening link: https://www.ebay.com/itm/276523158982
Page loaded successfully: https://www.ebay.com/itm/276523158982
Clicked 'See all feedback' button for 276523158982
Opening link: https://www.ebay.com/itm/276082722823
Page loaded successfully: https://www.ebay.com/itm/276082722823
Clicked 'See all feedback' button for 276082722823
Opening link: https://www.ebay.com/itm/204382636998
Page loaded successfully: https://www.ebay.com/itm/204382636998
Clicked 'See all feedback' button for 204382636998
Opening link: https://www.ebay.com/itm/155813861527
Page loaded successfully: https://www.ebay.com/itm/155813861527
Clicked 'See all feedback' button for 15581386152

In [402]:
# with open('feedback_data.csv', 'r', encoding='utf8') as file:
#     df = pd.read_csv(file)


In [455]:
df

,Item ID,Title,Price,Labels,Values,Feedback
0,276429448744,Samsung Galaxy S22 S22+ S22 Ultra 5G - 128GB -...,US $229.95,"[Shipping:, Import charges: , Delivery:, Retur...",[US $28.10 eBay International Shipping . See d...,[got refunded. Item was pretty worn on the out...
1,256193691993,Apple iPhone X | XR | XS | XS Max - 64GB 128GB...,US $154.95,"[Shipping:, Import charges: , Delivery:, Retur...",[US $33.41 eBay International Shipping . See d...,"[I’m very satisfied with my phone, good qualit..."
2,276523158982,Google Pixel 6 | 6a | 6 Pro - 128GB 256GB 512G...,US $139.95,"[Shipping:, Import charges: , Delivery:, Retur...",[US $25.44 eBay International Shipping . See d...,[Phone came in good condition. Screen protecto...
3,276082722823,Apple iPhone 11 - 64GB 128GB - Unlocked Verizo...,US $194.95,"[Shipping:, Import charges: , Delivery:, Retur...",[US $27.07 eBay International Shipping . See d...,[Honestly was not expecting much but phone loo...
4,204382636998,SHARP AQUOS Keitai 601SH Softbank Unlocked SIM...,US $75.00/ea,"[Shipping:, Delivery:, Returns:, Condition, Se...",[Free Economy International Shipping. See deta...,[The phone just looks perfect. It's the aesthe...
5,155813861527,Nokia 3310 Navy blue Unlocked 2G GSM 900/1800 ...,US $27.30,"[Shipping:, Delivery:, Returns:, Condition, Se...",[Free Standard International Shipping. See det...,"[Muy buen producto y buena calidad, Very smoot..."
6,156322635487,Samsung Galaxy Folder 2 SM-G1600 Dual Sim 16GB...,US $89.00/ea,"[Shipping:, Delivery:, Returns:, Condition, Se...",[Free Standard Shipping from Greater China to ...,[the phone works perfectly and it arrived soon...
7,275343078481,Samsung Galaxy S10 S10e S10+ Plus 128GB 512GB ...,US $119.85,"[Shipping:, Import charges: , Delivery:, Retur...",[US $23.82 eBay International Shipping . See d...,[Phone came in timely manner and came as adver...
8,256194567959,Google Pixel 7 Pro 128GB Unlocked - Very Good,US $279.99,"[Shipping:, Import charges: , Delivery:, Retur...",[US $29.58 eBay International Shipping . See d...,[Este producto es renovado por Ebay pero funci...
9,404911877165,FOR SPECTRUM Motorola Razr+ Plus 40 Ultra XT23...,US $216.99,"[Shipping:, Import charges: , Delivery:, Retur...",[US $37.16 eBay International Shipping . See d...,"[This phone is like-new condition, better than..."


In [456]:
df.columns

Index(['Item ID', 'Title', 'Price', 'Labels', 'Values', 'Feedback'], dtype='object')

In [457]:
for item in df["Values"][0]:
    print(item)

US $28.10 eBay International Shipping . See details for shipping Located in: Spartanburg, SC, United States
This item may be subject to import charges and taxes on delivery
Estimated between Mon, Nov 11 and Mon, Dec 2 to 0001 Please note the delivery estimate is greater than 29 business days. Seller ships within 1 day after receiving cleared payment.
30 days returns. Buyer pays for return shipping. See details - for more information about returns
Very Good - Refurbished
“This listing offers a variety of choices, including Model, Network, and Color options. The devices ... Read more about the seller notes
Octa Core
6.1 in, 6.6 in, 6.8 in
None
N/A
Factory Unlocked
SM-S901U, SM-S906U, SM-S908U
Dual SIM (SIM + eSIM)
Samsung
AT&T, Consumer Cellular, Cricket Wireless, Metro, Mint Mobile, SIMPLE Mobile, Spectrum, T-Mobile, U.S. Cellular, Unlocked, Verizon, Xfinity, h2o
3G, 4G, 4G+, 5G, Bluetooth, LTE, USB Type-C, Wi-Fi
Bar
3D Depth Camera, 3D Depth Sensor, 3D Face Recognition, 4K Video Record

In [349]:
# Iterate through the DataFrame and print each label with its corresponding value

def zip_col(labels, values):
    dict = {}
    # df_new = pd.DataFrame({'Labels': labels, 'Values': values})
    for label, value in zip(labels, values):
        dict[label] = value
    return dict

In [458]:
def zip_col(labels, values):
    dict = {}
    # df_new = pd.DataFrame({'Labels': labels, 'Values': values})
    for i in range(len(labels)):
        dict[labels[i]] = values[i]
    return dict

In [450]:
for i in range(0, len(df["Labels"])):
    print(zip_col(df["Labels"][i], df["Values"][i]))

{'Shipping:': 'Free Standard Shipping from Greater China to worldwide. See details for shipping', 'Delivery:': 'Estimated between Wed, Oct 23 and Wed, Nov 13 to 0001 Please note the delivery estimate is greater than 16 business days. Please allow additional time if international delivery is subject to customs processing.', 'Returns:': '30 days returns. Seller pays for return shipping. See details - for more information about returns', 'Condition': 'Used', 'Seller Notes': '“Sealed,with all accessories,Very good condition,tested,working well.”', 'Brand': 'Apple', 'Screen Size': '3.5 in', 'Model': 'Apple iPhone 4s', 'Storage Capacity': '32 GB', 'Camera Resolution': '8.0 MP'}


In [459]:
new_data = {}
new_data_list = []
for i in range(0, len(df["Labels"])):
    data_item = {
    "Item ID":df["Item ID"][i],
    "Title":df["Title"][i],
    "Price":df["Price"][i],
    "Labels":df["Labels"][i],
    "Values":df["Values"][i],
    "Zip_data":zip_col(df["Labels"][i], df["Values"][i]),
    "Feedback":df["Feedback"][i],
    "Feedback_clean":list(filter(None, df["Feedback"][i]))}

    new_data_list.append(data_item)

In [460]:
new_data_list

[{'Item ID': '276429448744',
  'Title': 'Samsung Galaxy S22 S22+ S22 Ultra 5G - 128GB - Unlocked Verizon T-Mobile AT&T',
  'Price': 'US $229.95',
  'Labels': ['Shipping:',
   'Import charges: ',
   'Delivery:',
   'Returns:',
   'Condition',
   'Seller Notes',
   'Processor',
   'Screen Size',
   'Memory Card Type',
   'MPN',
   'Lock Status',
   'Model Number',
   'SIM Card Slot',
   'Brand',
   'Network',
   'Connectivity',
   'Style',
   'Features',
   'Operating System',
   'Storage Capacity',
   'Contract',
   'Camera Resolution',
   'RAM'],
  'Values': ['US $28.10 eBay International Shipping . See details for shipping Located in: Spartanburg, SC, United States',
   'This item may be subject to import charges and taxes on delivery',
   'Estimated between Mon, Nov 11 and Mon, Dec 2 to 0001 Please note the delivery estimate is greater than 29 business days. Seller ships within 1 day after receiving cleared payment.',
   '30 days returns. Buyer pays for return shipping. See details -

In [461]:
new_df = pd.DataFrame(new_data_list)
new_df.to_csv('raw_data.csv')

In [357]:
for i in range(1):
    print('label',len(df["Labels"][i]))
    print('value',len(df["Values"][i]))

label 38
value 37


In [380]:
new_df[new_df["Item ID"] == '126244818703'].to_csv('11111111.csv')